# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [76]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [77]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [78]:
# Import the necessary CSVs to Pandas DataFrames
# YOUR CODE HERE!
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
file_path = Path("Data/neighborhoods_coordinates.csv")
coord_data = pd.read_csv(file_path, index_col="Neighborhood")



- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [79]:
sfo_housing = sfo_data.groupby('year').mean()
sfo_data_units = sfo_data["housing_units"]

sale_price_sqr_foot = sfo_data['sale_price_sqr_foot']
gross_rent = sfo_data['gross_rent']

year_neighbor = sfo_data.groupby(['year','neighborhood']).mean()
year_neighbor = year_neighbor.drop(columns='housing_units')

expensive_ten = year_neighbor.groupby('neighborhood').mean().sort_values('sale_price_sqr_foot',                               ascending=False).head(10).drop(columns='gross_rent')
 
top_ten_year = sfo_data[sfo_data['neighborhood'].isin(expensive_ten.index)].drop(columns='housing_units')

exp_nb_per_yr=sfo_data[sfo_data["neighborhood"].isin(expensive_ten.index)]
ten_exp_nb_per_yr=exp_nb_per_yr.groupby(['neighborhood']).mean().reset_index()

mean_val= sfo_data.groupby(['neighborhood']).mean()
join_location = pd.concat([coord_data, mean_val], axis='columns', join='outer').dropna()
join_location.reset_index(inplace=True)
px.set_mapbox_access_token(map_box_api)

data_sunburst=exp_nb_per_yr.reset_index()

In [105]:


# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    
    sfo_unit_avg = sfo_data_units.hvplot.bar(x="year", 
                                            y="housing_units",
                                            rot=90, 
                                            ylim=(370000, 385000)
                                            ).opts(yformatter="%.0f", 
                                              title="Housing Units in Sanfrancisco from 2010 to 2016")
    return sfo_unit_avg

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
 
    plot_gross_rent = gross_rent.hvplot(label='Average Gross Rent by Year',
                                line_color= 'red',
                                ylabel= 'Average Gross Rent',
                                grid=True)
    return plot_gross_rent

def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    plot_sale_price_sqr_foot = sale_price_sqr_foot.hvplot(label='Average Price per sq ft by Year',                                                                       line_color= 'green', 
                                                ylabel= 'Price Per sqft',
                                                grid=True)
    return plot_sale_price_sqr_foot

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!

    
    y_n_price = year_neighbor.hvplot(groupby=['neighborhood'],
                label='Average Price/sqft',
                x="year",
                y='sale_price_sqr_foot',
                dynamic=False)
    y_n_rent = year_neighbor.hvplot(groupby=['neighborhood'],
                label="Average Monthly Rent",
                x="year",
                y='gross_rent',
                dynamic=False)
    return y_n_price + y_n_rent

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    # YOUR CODE HERE!
   
    ex_ten = expensive_ten.hvplot(kind='bar', 
                            rot=45,
                            ylim=(600, 900), 
                            title="10 Most Expensive Neighborhoods")
    return ex_ten

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # YOUR CODE HERE!
   
    compare_ten=top_ten_year.hvplot(kind="bar", 
                rot=90, 
                label='High cost of living ',                       
                ylabel= 'Average Price', 
                x='year', 
                xlabel="Yearly Rent vs Price/sqft",                    
                groupby='neighborhood', 
                y=(['sale_price_sqr_foot', 'gross_rent'])
                )

    return compare_ten

def parallel_coordinates():
    """Parallel Coordinates Plot."""

    # YOUR CODE HERE!
   
    par_cord= px.parallel_coordinates(ten_exp_nb_per_yr, 
                                        color=ten_exp_nb_per_yr.index, 
                                        width=1000, 
                                        height=500)
    return par_cord

def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    par_cat= px.parallel_categories(ten_exp_nb_per_yr, color="sale_price_sqr_foot")
    return par_cat


def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!
    
    map_box=px.scatter_mapbox(
                join_location, 
                lat="Lat", 
                lon="Lon", 
                size="gross_rent", 
                color="index",
                zoom=12)
    map_box.update_layout(mapbox_style="streets")
    
    return map_box

def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!
    
    sun_burst=px.sunburst(data_sunburst,
                path=['year','neighborhood'],
                values="sale_price_sqr_foot",
                color="gross_rent", 
                color_continuous_scale='RdBu')
    return sun_burst


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [117]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
"### San Francisco Housing Dashboard"

# Create a tab layout for the dashboard
# YOUR CODE HERE!



cost_column = pn.Column(
                "## San Francisco Average Housing Cost ",
                    housing_units_per_year(),
                    (average_gross_rent(),
                    average_sales_price()),
                    average_price_by_neighborhood(),
                    "# Top 10 mMst Expensive Neighbor Hoods",
                    top_most_expensive_neighborhoods(),
                    most_expensive_neighborhoods_rent_sales()

)

graphic_column = pn.Column(
                    "##Interactive Info Graphics",
                    parallel_categories(),
                    parallel_coordinates(),
                    neighborhood_map(),
                    sunburst()
)
# Create the dashboard
# YOUR CODE HERE!

sfo_dash = pn.Tabs(
        ("Cost", cost_column),("Interactive Info Graphics", graphic_column)
    )

## Serve the Panel Dashboard

In [118]:
# Serve the# dashboard
# YOUR CODE HERE!
sfo_dash.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] Str(tuple)
        [3] Row
            [0] HoloViews(Layout)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [4] Markdown(str)
        [5] HoloViews(Bars)
        [6] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Cow Hollow', ...], value='Cow Hollow', width=250)
                [1] VSpacer()
    [1] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
        [3] Plotly(Figure)
        [4] Plotly(Figure)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [80]:
 housing_units_per_year()

:Bars   [year]   (housing_units)

In [81]:
average_gross_rent()

:Curve   [year]   (gross_rent)

In [106]:
 average_sales_price()

:Curve   [year]   (sale_price_sqr_foot)

In [85]:
average_price_by_neighborhood()

:Layout
   .HoloMap.Average_Price_over_sqft :HoloMap   [neighborhood]
      :Curve   [year]   (sale_price_sqr_foot)
   .HoloMap.Average_Monthly_Rent    :HoloMap   [neighborhood]
      :Curve   [year]   (gross_rent)

In [86]:
top_most_expensive_neighborhoods()

:Bars   [neighborhood]   (sale_price_sqr_foot)

In [89]:
most_expensive_neighborhoods_rent_sales()

:DynamicMap   [neighborhood]
   :Bars   [year,Variable]   (value)

In [90]:
neighborhood_map().show()

In [94]:
 parallel_categories()

In [95]:
parallel_coordinates()

In [96]:
sunburst()